introduction:

In [3]:
#code packages
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [30]:
#preliminary data: https://archive.ics.uci.edu/dataset/186/wine+quality

#data
red_wine <- read_delim('data/winequality-red.csv', delim = ';')
red_wine <- red_wine |>
mutate(quality = as_factor(quality))
colnames(red_wine) <- c("fixed_acidity","volatile_acidity","citric_acid","residual_sugar", "chlorides",
"free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol", "quality")
# colnames(red_wine)
# head(red_wine)

#general training data
wine_split <- initial_split(red_wine, prop = 0.75, strata = quality)
wine_train <- training(wine_split)
wine_test <- testing(wine_split)
# wine_train
# wine_test

#choosing random predictors lol, using training data
wine_recipe <- recipe(quality ~ fixed_acidity + pH + alcohol, data = wine_train) |>
step_scale(all_predictors()) |>
step_center(all_predictors())
#wine_recipe

wine_spec <- nearest_neighbor(weight_func = 'rectangular', neighbors = 5) |>
set_engine('kknn') |>
set_mode('classification')


wine_fit <- workflow() |>
add_recipe(wine_recipe) |>
add_model(wine_spec)|>
fit(data = wine_train)

wine_predict <- predict(wine_fit, wine_test) |>
bind_cols(wine_test)
#wine_predict

wine_accuracy <- wine_predict |>
metrics(truth = quality, estimate = .pred_class)
wine_accuracy

wine_mat <- wine_predict |>
conf_mat(truth = quality, estimate = .pred_class)
wine_mat

Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (12): fixed acidity, volatile acidity, citric acid, residual sugar, chlo...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.5236908
kap,multiclass,0.2319471


          Truth
Prediction   3   4   5   6   7   8
         3   0   0   0   0   0   0
         4   0   0   0   0   0   0
         5   1   9 112  55   9   1
         6   0   7  52  83  24   1
         7   0   0   5  25  15   2
         8   0   0   0   0   0   0

methods:

expected outcome and significance: